In [1]:
import pandas as pd
from datetime import timedelta
from arcgis.features import GeoAccessor, GeoSeriesAccessor
import matplotlib.pyplot as plt

In [2]:
#set filepaths for data

apr_csv = r"G:\MGIS\Spatiotemporal Modeling\Final Project\apc_data\dat_202204.csv"
jul_csv = r"G:\MGIS\Spatiotemporal Modeling\Final Project\apc_data\dat_202207.csv"
oct_csv = r"G:\MGIS\Spatiotemporal Modeling\Final Project\apc_data\dat_202210.csv"
feb_csv = r"G:\MGIS\Spatiotemporal Modeling\Final Project\apc_data\dat_202302.csv"

In [3]:
#read april as df
apr_df = pd.read_csv(apr_csv)

apr_df


,board,alight,c_load,calendar_id,block_abbr,trip_id,route_abbr,line_type,o_stop,stop_sequence_number,service_abbr,route_direction_abbr,pattern_abbr,block_stop_order,latitude,longitude,actual_arrival_time
0,1,0,3,20220401,1462,1,54,Urb Loc,11861,5,WK,W,UNDPMOA200,6,44.947217,-93.093799,11156
1,0,0,3,20220401,1462,1,54,Urb Loc,11862,6,WK,W,UNDPMOA200,7,44.946197,-93.095645,11240
2,0,0,3,20220401,1462,1,54,Urb Loc,11863,7,WK,W,UNDPMOA200,8,44.946112,-93.097146,11256
3,0,0,3,20220401,1462,1,54,Urb Loc,3264,8,WK,W,UNDPMOA200,9,44.946248,-93.099029,11268
4,0,0,3,20220401,1462,1,54,Urb Loc,47485,9,WK,W,UNDPMOA200,10,44.945413,-93.102246,11360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119328,0,0,1,20220430,1391,141,724,Sub Loc,9460,33,SAT,S,STL1BCTC00,755,45.068471,-93.319445,92244
8119329,0,0,1,20220430,1391,141,724,Sub Loc,9464,34,SAT,S,STL1BCTC00,756,45.066872,-93.319290,92256
8119330,0,0,1,20220430,1391,141,724,Sub Loc,9468,35,SAT,S,STL1BCTC00,757,45.064253,-93.319230,92276
8119331,0,0,1,20220430,1391,141,724,Sub Loc,9470,36,SAT,S,STL1BCTC00,758,45.062241,-93.319271,92292


In [4]:
#read other months as df
jul_df = pd.read_csv(jul_csv)
oct_df = pd.read_csv(oct_csv)
feb_df = pd.read_csv(feb_csv)

#jul_df

## Time bucket creation

In [5]:
# Function to round up to the next 30-minute bucket - created with assistance of ChatGPT, Open AI
def round_to_next_30_minutes(time_str):
    # Ensure time is always in HH:MM:SS format
    hours, minutes, seconds = map(int, time_str.split(':'))
    time_obj = timedelta(hours=hours, minutes=minutes, seconds=seconds)
    
    # Calculate total minutes since midnight
    total_minutes = time_obj.seconds // 60
    
    # Calculate the next 30-minute bucket
    next_bucket_minutes = (total_minutes + 29) // 30 * 30
    
    # Convert the minutes back to a timedelta object
    rounded_time = timedelta(minutes=next_bucket_minutes)
    
    # Format the rounded time as HH:MM:SS
    rounded_hours = rounded_time.seconds // 3600
    rounded_minutes = (rounded_time.seconds % 3600) // 60
    rounded_seconds = rounded_time.seconds % 60

    return f"{rounded_hours:02}:{rounded_minutes:02}:{rounded_seconds:02}"

## April Weekdays

In [6]:
#convert time to known time format
apr_df["time"] = apr_df["actual_arrival_time"].apply(lambda x: str(timedelta(seconds=x % 86400)))

apr_df

,board,alight,c_load,calendar_id,block_abbr,trip_id,route_abbr,line_type,o_stop,stop_sequence_number,service_abbr,route_direction_abbr,pattern_abbr,block_stop_order,latitude,longitude,actual_arrival_time,time
0,1,0,3,20220401,1462,1,54,Urb Loc,11861,5,WK,W,UNDPMOA200,6,44.947217,-93.093799,11156,3:05:56
1,0,0,3,20220401,1462,1,54,Urb Loc,11862,6,WK,W,UNDPMOA200,7,44.946197,-93.095645,11240,3:07:20
2,0,0,3,20220401,1462,1,54,Urb Loc,11863,7,WK,W,UNDPMOA200,8,44.946112,-93.097146,11256,3:07:36
3,0,0,3,20220401,1462,1,54,Urb Loc,3264,8,WK,W,UNDPMOA200,9,44.946248,-93.099029,11268,3:07:48
4,0,0,3,20220401,1462,1,54,Urb Loc,47485,9,WK,W,UNDPMOA200,10,44.945413,-93.102246,11360,3:09:20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119328,0,0,1,20220430,1391,141,724,Sub Loc,9460,33,SAT,S,STL1BCTC00,755,45.068471,-93.319445,92244,1:37:24
8119329,0,0,1,20220430,1391,141,724,Sub Loc,9464,34,SAT,S,STL1BCTC00,756,45.066872,-93.319290,92256,1:37:36
8119330,0,0,1,20220430,1391,141,724,Sub Loc,9468,35,SAT,S,STL1BCTC00,757,45.064253,-93.319230,92276,1:37:56
8119331,0,0,1,20220430,1391,141,724,Sub Loc,9470,36,SAT,S,STL1BCTC00,758,45.062241,-93.319271,92292,1:38:12


In [7]:
#Apply the function to create a new column
apr_df['time_bucket'] = apr_df['time'].apply(round_to_next_30_minutes)

# Output the DataFrame to check the new column
#apr_df

In [8]:
#split into two dataframes for weekday and weekends

apr_wk = apr_df[apr_df['service_abbr'] == 'WK']
#apr_sat_sun = apr_df[apr_df['service_abbr'].isin(['SAT', 'SUN'])]

apr_wk
#apr_sat_sun

,board,alight,c_load,calendar_id,block_abbr,trip_id,route_abbr,line_type,o_stop,stop_sequence_number,service_abbr,route_direction_abbr,pattern_abbr,block_stop_order,latitude,longitude,actual_arrival_time,time,time_bucket
0,1,0,3,20220401,1462,1,54,Urb Loc,11861,5,WK,W,UNDPMOA200,6,44.947217,-93.093799,11156,3:05:56,03:30:00
1,0,0,3,20220401,1462,1,54,Urb Loc,11862,6,WK,W,UNDPMOA200,7,44.946197,-93.095645,11240,3:07:20,03:30:00
2,0,0,3,20220401,1462,1,54,Urb Loc,11863,7,WK,W,UNDPMOA200,8,44.946112,-93.097146,11256,3:07:36,03:30:00
3,0,0,3,20220401,1462,1,54,Urb Loc,3264,8,WK,W,UNDPMOA200,9,44.946248,-93.099029,11268,3:07:48,03:30:00
4,0,0,3,20220401,1462,1,54,Urb Loc,47485,9,WK,W,UNDPMOA200,10,44.945413,-93.102246,11360,3:09:20,03:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7888202,1,1,7,20220429,1621,370,3,Urb Loc,57018,65,WK,E,WA105CED00,551,NaN,NaN,94410,2:13:30,02:30:00
7888203,0,1,6,20220429,1621,370,3,Urb Loc,49450,66,WK,E,WA105CED00,552,44.950081,-93.100056,94516,2:15:16,02:30:00
7888204,0,0,6,20220429,1621,370,3,Urb Loc,11912,67,WK,E,WA105CED00,553,44.950506,-93.097629,94598,2:16:38,02:30:00
7888205,0,0,6,20220429,1621,370,3,Urb Loc,20013,68,WK,E,WA105CED00,554,44.947507,-93.094443,94696,2:18:16,02:30:00


In [48]:
print(apr_wk['line_type'].unique())

['Urb Loc' 'Sub Loc']


In [43]:
# Group by time_bucket and calendar_id, then count rows for each combination
apr_grouped = apr_wk.groupby(['time_bucket', 'calendar_id']).size().reset_index(name='count')

# Calculate the average count per time_bucket
apr_avg_counts = apr_grouped.groupby('time_bucket')['count'].mean()

# Sort the time_bucket values with 03:30:00 as the starting point
apr_avg_counts_sorted = apr_avg_counts.sort_index()  # Sort by time buckets
apr_reordered_avg_counts = pd.concat([
    apr_avg_counts_sorted.loc["03:30:00":],  # Start from 03:30:00 onward
    apr_avg_counts_sorted.loc[:"03:00:00"]   # Append 00:00:00 to 03:00:00 at the end
])

# Plot the reordered average histogram for April
plt.figure(figsize=(10, 6))
apr_reordered_avg_counts.plot(kind='bar', color='skyblue')

# Add title and labels for the chart
plt.title('Average Count of Stops per Time Bucket in April - Weekday', fontsize=14)
plt.xlabel('Time Bucket', fontsize=12)
plt.ylabel('Average Count', fontsize=12)

# Rotate the x-axis labels for readability
plt.xticks(rotation=45, ha='right')

# Show the plot
plt.tight_layout()
plt.show()

In [41]:
#summary statistics for board and alight per stop per time bucket
#could use alternate method for picking lat/lon, but they should be the same for each stop, I just need to retain it

apr_wk_summ = apr_wk.groupby(["o_stop", "time_bucket"]).agg({
    "board": "sum",
    "alight": "sum",
    "latitude": "min",
    "longitude": "min"
}).reset_index()

In [42]:
apr_wk_summ

,o_stop,time_bucket,board,alight,latitude,longitude
0,34,00:00:00,28,8,44.971011,-93.262521
1,34,00:30:00,23,0,44.971011,-93.262521
2,34,01:00:00,1,0,44.971011,-93.262521
3,34,01:30:00,22,10,44.971011,-93.262521
4,34,04:30:00,13,3,44.971011,-93.262521
...,...,...,...,...,...,...
239453,90241,14:00:00,0,0,NaN,NaN
239454,90241,15:00:00,0,0,NaN,NaN
239455,90241,16:00:00,0,0,NaN,NaN
239456,90241,17:00:00,0,0,NaN,NaN


In [44]:
apr_wk_summ['dummy_timestamp'] = '2022/04/01 '+ apr_wk_summ['time_bucket'] 

apr_wk_sedf = pd.DataFrame.spatial.from_xy(apr_wk_summ, x_column="longitude", y_column="latitude", sr=4326)

apr_wk_sedf

,o_stop,time_bucket,board,alight,latitude,longitude,dummy_timestamp,SHAPE
0,34,00:00:00,28,8,44.971011,-93.262521,2022/04/01 00:00:00,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
1,34,00:30:00,23,0,44.971011,-93.262521,2022/04/01 00:30:00,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
2,34,01:00:00,1,0,44.971011,-93.262521,2022/04/01 01:00:00,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
3,34,01:30:00,22,10,44.971011,-93.262521,2022/04/01 01:30:00,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
4,34,04:30:00,13,3,44.971011,-93.262521,2022/04/01 04:30:00,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
...,...,...,...,...,...,...,...,...
239453,90241,14:00:00,0,0,NaN,NaN,2022/04/01 14:00:00,None
239454,90241,15:00:00,0,0,NaN,NaN,2022/04/01 15:00:00,None
239455,90241,16:00:00,0,0,NaN,NaN,2022/04/01 16:00:00,None
239456,90241,17:00:00,0,0,NaN,NaN,2022/04/01 17:00:00,None


In [45]:
#convert to spatial dataframe and then to feature class
#apr_wk_sedf = pd.DataFrame.spatial.from_xy(apr_wk_summ, x_column="longitude", y_column="latitude", sr=4326)

apr_wk_sedf.spatial.to_featureclass(location = r"C:\Users\KOlso\Documents\ArcGIS\Projects\SpatioTempporal Modeling Final Project\SpatioTempporal Modeling Final Project.gdb\apr_wk_summary")


'C:\\Users\\KOlso\\Documents\\ArcGIS\\Projects\\SpatioTempporal Modeling Final Project\\SpatioTempporal Modeling Final Project.gdb\\apr_wk_summary'

In [46]:
arcpy.management.ConvertTimeField(
    in_table="apr_wk_summary",
    input_time_field="dummy_timestamp",
    input_time_format="yyyy/MM/dd HH:mm:ss",
    output_time_field="dummy_timestamp_Converted",
    output_time_type="DATE",
    output_time_format="",
    timezone_or_field=""
)

<Result 'apr_wk_summary'>

## July - weekdays

In [9]:
#convert time to known time format
jul_df["time"] = jul_df["actual_arrival_time"].apply(lambda x: str(timedelta(seconds=x % 86400)))

jul_df

,board,alight,c_load,calendar_id,block_abbr,trip_id,route_abbr,line_type,o_stop,stop_sequence_number,service_abbr,route_direction_abbr,pattern_abbr,block_stop_order,latitude,longitude,actual_arrival_time,time
0,1,0,1,20220701,1424,1,54,Urb Loc,56100,1,WK,W,UNDPMOA200,2,44.947154,-93.085622,10188,2:49:48
1,0,0,1,20220701,1424,1,54,Urb Loc,56976,2,WK,W,UNDPMOA200,3,NaN,NaN,10974,3:02:54
2,0,0,1,20220701,1424,1,54,Urb Loc,11858,3,WK,W,UNDPMOA200,4,44.949291,-93.089339,11050,3:04:10
3,0,0,1,20220701,1424,1,54,Urb Loc,11859,4,WK,W,UNDPMOA200,5,44.948555,-93.091098,11080,3:04:40
4,0,0,1,20220701,1424,1,54,Urb Loc,11861,5,WK,W,UNDPMOA200,6,44.947217,-93.093799,11110,3:05:10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8077162,0,0,0,20220731,1278,166,64,Urb Loc,10419,46,SUN,E,5WASHILL54,641,44.984570,-93.025146,92732,1:45:32
8077163,0,0,0,20220731,1278,166,64,Urb Loc,40014,47,SUN,E,5WASHILL54,642,44.986401,-93.025091,92746,1:45:46
8077164,0,0,0,20220731,1278,166,64,Urb Loc,51604,48,SUN,E,5WASHILL54,643,44.988605,-93.025166,92766,1:46:06
8077165,0,0,0,20220731,1278,166,64,Urb Loc,40007,49,SUN,E,5WASHILL54,644,44.989840,-93.025126,92776,1:46:16


In [10]:
# Apply the function to create a new column
jul_df['time_bucket'] = jul_df['time'].apply(round_to_next_30_minutes)

# Output the DataFrame to check the new column
jul_df

,board,alight,c_load,calendar_id,block_abbr,trip_id,route_abbr,line_type,o_stop,stop_sequence_number,service_abbr,route_direction_abbr,pattern_abbr,block_stop_order,latitude,longitude,actual_arrival_time,time,time_bucket
0,1,0,1,20220701,1424,1,54,Urb Loc,56100,1,WK,W,UNDPMOA200,2,44.947154,-93.085622,10188,2:49:48,03:00:00
1,0,0,1,20220701,1424,1,54,Urb Loc,56976,2,WK,W,UNDPMOA200,3,NaN,NaN,10974,3:02:54,03:30:00
2,0,0,1,20220701,1424,1,54,Urb Loc,11858,3,WK,W,UNDPMOA200,4,44.949291,-93.089339,11050,3:04:10,03:30:00
3,0,0,1,20220701,1424,1,54,Urb Loc,11859,4,WK,W,UNDPMOA200,5,44.948555,-93.091098,11080,3:04:40,03:30:00
4,0,0,1,20220701,1424,1,54,Urb Loc,11861,5,WK,W,UNDPMOA200,6,44.947217,-93.093799,11110,3:05:10,03:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8077162,0,0,0,20220731,1278,166,64,Urb Loc,10419,46,SUN,E,5WASHILL54,641,44.984570,-93.025146,92732,1:45:32,02:00:00
8077163,0,0,0,20220731,1278,166,64,Urb Loc,40014,47,SUN,E,5WASHILL54,642,44.986401,-93.025091,92746,1:45:46,02:00:00
8077164,0,0,0,20220731,1278,166,64,Urb Loc,51604,48,SUN,E,5WASHILL54,643,44.988605,-93.025166,92766,1:46:06,02:00:00
8077165,0,0,0,20220731,1278,166,64,Urb Loc,40007,49,SUN,E,5WASHILL54,644,44.989840,-93.025126,92776,1:46:16,02:00:00


In [11]:
jul_wk = jul_df[jul_df['service_abbr'] == 'WK']
#apr_sat_sun = apr_df[apr_df['service_abbr'].isin(['SAT', 'SUN'])]

jul_wk

,board,alight,c_load,calendar_id,block_abbr,trip_id,route_abbr,line_type,o_stop,stop_sequence_number,service_abbr,route_direction_abbr,pattern_abbr,block_stop_order,latitude,longitude,actual_arrival_time,time,time_bucket
0,1,0,1,20220701,1424,1,54,Urb Loc,56100,1,WK,W,UNDPMOA200,2,44.947154,-93.085622,10188,2:49:48,03:00:00
1,0,0,1,20220701,1424,1,54,Urb Loc,56976,2,WK,W,UNDPMOA200,3,NaN,NaN,10974,3:02:54,03:30:00
2,0,0,1,20220701,1424,1,54,Urb Loc,11858,3,WK,W,UNDPMOA200,4,44.949291,-93.089339,11050,3:04:10,03:30:00
3,0,0,1,20220701,1424,1,54,Urb Loc,11859,4,WK,W,UNDPMOA200,5,44.948555,-93.091098,11080,3:04:40,03:30:00
4,0,0,1,20220701,1424,1,54,Urb Loc,11861,5,WK,W,UNDPMOA200,6,44.947217,-93.093799,11110,3:05:10,03:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7670641,0,0,4,20220729,1067,370,3,Urb Loc,57018,65,WK,E,WA105CED00,341,NaN,NaN,94146,2:09:06,02:30:00
7670642,0,0,4,20220729,1067,370,3,Urb Loc,49450,66,WK,E,WA105CED00,342,44.950081,-93.100056,94186,2:09:46,02:30:00
7670643,0,1,3,20220729,1067,370,3,Urb Loc,11912,67,WK,E,WA105CED00,343,44.950506,-93.097629,94238,2:10:38,02:30:00
7670644,0,1,2,20220729,1067,370,3,Urb Loc,20013,68,WK,E,WA105CED00,344,44.947507,-93.094443,94288,2:11:28,02:30:00


In [15]:
# --- JULY ---
# Group by time_bucket and calendar_id, then count rows for each combination
jul_grouped = jul_wk.groupby(['time_bucket', 'calendar_id']).size().reset_index(name='count')

# Calculate the average count per time_bucket
jul_avg_counts = jul_grouped.groupby('time_bucket')['count'].mean()

# Sort the time_bucket values with 03:30:00 as the starting point
jul_avg_counts_sorted = jul_avg_counts.sort_index()  # Sort by time buckets
jul_reordered_avg_counts = pd.concat([
    jul_avg_counts_sorted.loc["03:30:00":],  # Start from 03:30:00 onward
    jul_avg_counts_sorted.loc[:"03:00:00"]   # Append 00:00:00 to 03:00:00 at the end
])

# Plot the reordered average histogram for July
plt.figure(figsize=(10, 6))
jul_reordered_avg_counts.plot(kind='bar', color='skyblue')

# Add title and labels for the chart
plt.title('Average Count of Stops per Time Bucket in July - Weekday', fontsize=14)
plt.xlabel('Time Bucket', fontsize=12)
plt.ylabel('Average Count', fontsize=12)

# Rotate the x-axis labels for readability
plt.xticks(rotation=45, ha='right')

# Show the plot
plt.tight_layout()
plt.show()

In [53]:
jul_wk_summ = jul_wk.groupby(["o_stop", "time_bucket"]).agg({
    "board": "sum",
    "alight": "sum",
    "latitude": "min",
    "longitude": "min"
}).reset_index()

In [54]:
jul_wk_summ

,o_stop,time_bucket,board,alight,latitude,longitude
0,34,00:00:00,32,15,44.971011,-93.262521
1,34,00:30:00,33,13,44.971011,-93.262521
2,34,01:30:00,19,13,44.971011,-93.262521
3,34,04:30:00,6,0,44.971011,-93.262521
4,34,05:30:00,16,5,44.971011,-93.262521
...,...,...,...,...,...,...
238819,90241,14:00:00,0,0,NaN,NaN
238820,90241,15:00:00,0,0,NaN,NaN
238821,90241,16:00:00,0,0,NaN,NaN
238822,90241,17:00:00,0,0,NaN,NaN


In [55]:
jul_wk_summ['dummy_timestamp'] = '2022/07/01 '+ jul_wk_summ['time_bucket'] 

jul_wk_sedf = pd.DataFrame.spatial.from_xy(jul_wk_summ, x_column="longitude", y_column="latitude", sr=4326)

jul_wk_sedf

,o_stop,time_bucket,board,alight,latitude,longitude,dummy_timestamp,SHAPE
0,34,00:00:00,32,15,44.971011,-93.262521,2022/04/01 00:00:00,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
1,34,00:30:00,33,13,44.971011,-93.262521,2022/04/01 00:30:00,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
2,34,01:30:00,19,13,44.971011,-93.262521,2022/04/01 01:30:00,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
3,34,04:30:00,6,0,44.971011,-93.262521,2022/04/01 04:30:00,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
4,34,05:30:00,16,5,44.971011,-93.262521,2022/04/01 05:30:00,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
...,...,...,...,...,...,...,...,...
238819,90241,14:00:00,0,0,NaN,NaN,2022/04/01 14:00:00,None
238820,90241,15:00:00,0,0,NaN,NaN,2022/04/01 15:00:00,None
238821,90241,16:00:00,0,0,NaN,NaN,2022/04/01 16:00:00,None
238822,90241,17:00:00,0,0,NaN,NaN,2022/04/01 17:00:00,None


In [56]:
jul_wk_sedf.spatial.to_featureclass(location = r"C:\Users\KOlso\Documents\ArcGIS\Projects\SpatioTempporal Modeling Final Project\SpatioTempporal Modeling Final Project.gdb\jul_wk_summary")


'C:\\Users\\KOlso\\Documents\\ArcGIS\\Projects\\SpatioTempporal Modeling Final Project\\SpatioTempporal Modeling Final Project.gdb\\jul_wk_summary'

In [57]:
arcpy.management.ConvertTimeField(
    in_table="jul_wk_summary",
    input_time_field="dummy_timestamp",
    input_time_format="yyyy/MM/dd HH:mm:ss",
    output_time_field="dummy_timestamp_Converted",
    output_time_type="DATE",
    output_time_format="",
    timezone_or_field=""
)

<Result 'jul_wk_summary'>

In [58]:
arcpy.management.ApplySymbologyFromLayer(
    in_layer="jul_wk_summary",
    in_symbology_layer="apr_wk_summary",
    symbology_fields="VALUE_FIELD board board",
    update_symbology="MAINTAIN"
)

<Result 'jul_wk_summary'>

## July - Route 2 and 3

In [53]:
#set up route 2 and 3 layers now
jul_wk_3 = jul_wk[jul_wk['route_abbr'] == 3].copy()
jul_wk_2 = jul_wk[jul_wk['route_abbr'] == 2].copy()

In [54]:
jul_wk_3 = jul_wk_3.groupby(["o_stop", "time_bucket"]).agg({
    "board": "sum",
    "alight": "sum",
    "latitude": "min",
    "longitude": "min"
}).reset_index()

jul_wk_2 = jul_wk_2.groupby(["o_stop", "time_bucket"]).agg({
    "board": "sum",
    "alight": "sum",
    "latitude": "min",
    "longitude": "min"
}).reset_index()

In [55]:
jul_wk_3['dummy_timestamp'] = '2022/04/01 '+ jul_wk_3['time_bucket'] 

jul_wk_2['dummy_timestamp'] = '2022/04/01 '+ jul_wk_2['time_bucket'] 

In [56]:
jul_wk_3_sedf = pd.DataFrame.spatial.from_xy(jul_wk_3, x_column="longitude", y_column="latitude", sr=4326)
jul_wk_2_sedf = pd.DataFrame.spatial.from_xy(jul_wk_2, x_column="longitude", y_column="latitude", sr=4326)

jul_wk_3_sedf

,o_stop,time_bucket,board,alight,latitude,longitude,dummy_timestamp,SHAPE
0,826,00:30:00,0,1,44.960589,-93.108782,2022/04/01 00:30:00,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
1,826,05:30:00,0,0,44.960589,-93.108782,2022/04/01 05:30:00,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
2,826,06:30:00,0,0,44.960589,-93.108782,2022/04/01 06:30:00,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
3,826,07:00:00,1,0,44.960589,-93.108782,2022/04/01 07:00:00,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
4,826,07:30:00,0,2,44.960589,-93.108782,2022/04/01 07:30:00,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
...,...,...,...,...,...,...,...,...
7264,80377,21:30:00,5,6,NaN,NaN,2022/04/01 21:30:00,None
7265,80377,22:00:00,11,6,NaN,NaN,2022/04/01 22:00:00,None
7266,80377,22:30:00,0,0,NaN,NaN,2022/04/01 22:30:00,None
7267,80377,23:00:00,4,0,NaN,NaN,2022/04/01 23:00:00,None


In [57]:
jul_wk_3_sedf.spatial.to_featureclass(location = r"C:\Users\KOlso\Documents\ArcGIS\Projects\SpatioTempporal Modeling Final Project\SpatioTempporal Modeling Final Project.gdb\jul_wk_3_summary")
jul_wk_2_sedf.spatial.to_featureclass(location = r"C:\Users\KOlso\Documents\ArcGIS\Projects\SpatioTempporal Modeling Final Project\SpatioTempporal Modeling Final Project.gdb\jul_wk_2_summary")


'C:\\Users\\KOlso\\Documents\\ArcGIS\\Projects\\SpatioTempporal Modeling Final Project\\SpatioTempporal Modeling Final Project.gdb\\jul_wk_2_summary'

## October - weekdays

In [12]:
#convert time to known time format
oct_df["time"] = oct_df["actual_arrival_time"].apply(lambda x: str(timedelta(seconds=x % 86400)))

oct_df

,board,alight,c_load,calendar_id,block_abbr,trip_id,route_abbr,line_type,o_stop,stop_sequence_number,service_abbr,route_direction_abbr,pattern_abbr,block_stop_order,latitude,longitude,actual_arrival_time,time
0,0,0,2,20221001,1281,1,54,Local,10358,11,SAT,W,UNDPMOA200,12,44.942673,-93.105112,11454,3:10:54
1,0,0,2,20221001,1281,1,54,Local,10359,12,SAT,W,UNDPMOA200,13,44.941437,-93.106737,11470,3:11:10
2,0,0,2,20221001,1281,1,54,Local,10366,13,SAT,W,UNDPMOA200,14,44.934241,-93.117310,11554,3:12:34
3,0,0,2,20221001,1281,1,54,Local,11698,14,SAT,W,UNDPMOA200,15,44.926972,-93.127943,11672,3:14:32
4,0,0,2,20221001,1281,1,54,Local,13117,15,SAT,W,UNDPMOA200,16,44.923360,-93.133283,11714,3:15:14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7969556,0,3,-3,20221031,1066,370,3,Local,57018,65,WK,E,WALA5CED00,407,NaN,NaN,94390,2:13:10
7969557,0,0,-3,20221031,1066,370,3,Local,49450,66,WK,E,WALA5CED00,408,44.950081,-93.100056,94498,2:14:58
7969558,0,0,-3,20221031,1066,370,3,Local,11912,67,WK,E,WALA5CED00,409,44.950506,-93.097629,94538,2:15:38
7969559,0,0,-3,20221031,1066,370,3,Local,20013,68,WK,E,WALA5CED00,410,44.947507,-93.094443,94626,2:17:06


In [13]:
# Apply the function to create a new column
oct_df['time_bucket'] = oct_df['time'].apply(round_to_next_30_minutes)

# Output the DataFrame to check the new column
oct_df

,board,alight,c_load,calendar_id,block_abbr,trip_id,route_abbr,line_type,o_stop,stop_sequence_number,service_abbr,route_direction_abbr,pattern_abbr,block_stop_order,latitude,longitude,actual_arrival_time,time,time_bucket
0,0,0,2,20221001,1281,1,54,Local,10358,11,SAT,W,UNDPMOA200,12,44.942673,-93.105112,11454,3:10:54,03:30:00
1,0,0,2,20221001,1281,1,54,Local,10359,12,SAT,W,UNDPMOA200,13,44.941437,-93.106737,11470,3:11:10,03:30:00
2,0,0,2,20221001,1281,1,54,Local,10366,13,SAT,W,UNDPMOA200,14,44.934241,-93.117310,11554,3:12:34,03:30:00
3,0,0,2,20221001,1281,1,54,Local,11698,14,SAT,W,UNDPMOA200,15,44.926972,-93.127943,11672,3:14:32,03:30:00
4,0,0,2,20221001,1281,1,54,Local,13117,15,SAT,W,UNDPMOA200,16,44.923360,-93.133283,11714,3:15:14,03:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7969556,0,3,-3,20221031,1066,370,3,Local,57018,65,WK,E,WALA5CED00,407,NaN,NaN,94390,2:13:10,02:30:00
7969557,0,0,-3,20221031,1066,370,3,Local,49450,66,WK,E,WALA5CED00,408,44.950081,-93.100056,94498,2:14:58,02:30:00
7969558,0,0,-3,20221031,1066,370,3,Local,11912,67,WK,E,WALA5CED00,409,44.950506,-93.097629,94538,2:15:38,02:30:00
7969559,0,0,-3,20221031,1066,370,3,Local,20013,68,WK,E,WALA5CED00,410,44.947507,-93.094443,94626,2:17:06,02:30:00


In [14]:
oct_wk = oct_df[oct_df['service_abbr'] == 'WK']
#apr_sat_sun = apr_df[apr_df['service_abbr'].isin(['SAT', 'SUN'])]

oct_wk

,board,alight,c_load,calendar_id,block_abbr,trip_id,route_abbr,line_type,o_stop,stop_sequence_number,service_abbr,route_direction_abbr,pattern_abbr,block_stop_order,latitude,longitude,actual_arrival_time,time,time_bucket
382003,1,0,1,20221003,1440,1,54,Local,56100,1,WK,W,UNDPMOA200,2,44.947154,-93.085622,10518,2:55:18,03:00:00
382004,0,0,1,20221003,1440,1,54,Local,56976,2,WK,W,UNDPMOA200,3,NaN,NaN,10894,3:01:34,03:30:00
382005,0,0,1,20221003,1440,1,54,Local,11858,3,WK,W,UNDPMOA200,4,44.949291,-93.089339,10938,3:02:18,03:30:00
382006,2,0,3,20221003,1440,1,54,Local,11859,4,WK,W,UNDPMOA200,5,44.948555,-93.091098,10950,3:02:30,03:30:00
382007,2,0,5,20221003,1440,1,54,Local,11861,5,WK,W,UNDPMOA200,6,44.947217,-93.093799,11042,3:04:02,03:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7969556,0,3,-3,20221031,1066,370,3,Local,57018,65,WK,E,WALA5CED00,407,NaN,NaN,94390,2:13:10,02:30:00
7969557,0,0,-3,20221031,1066,370,3,Local,49450,66,WK,E,WALA5CED00,408,44.950081,-93.100056,94498,2:14:58,02:30:00
7969558,0,0,-3,20221031,1066,370,3,Local,11912,67,WK,E,WALA5CED00,409,44.950506,-93.097629,94538,2:15:38,02:30:00
7969559,0,0,-3,20221031,1066,370,3,Local,20013,68,WK,E,WALA5CED00,410,44.947507,-93.094443,94626,2:17:06,02:30:00


In [30]:
# Group by time_bucket and calendar_id, then count rows for each combination
oct_grouped = oct_wk.groupby(['time_bucket', 'calendar_id']).size().reset_index(name='count')

# Calculate the average count per time_bucket
oct_avg_counts = oct_grouped.groupby('time_bucket')['count'].mean()

# Sort the time_bucket values with 03:30:00 as the starting point
oct_avg_counts_sorted = oct_avg_counts.sort_index()  # Sort by time buckets
oct_reordered_avg_counts = pd.concat([
    oct_avg_counts_sorted.loc["03:30:00":],  # Start from 03:30:00 onward
    oct_avg_counts_sorted.loc[:"03:00:00"]   # Append 00:00:00 to 03:00:00 at the end
])

# Plot the reordered average histogram for October
plt.figure(figsize=(10, 6))
oct_reordered_avg_counts.plot(kind='bar', color='skyblue')

# Add title and labels for the chart
plt.title('Average Count of Stops per Time Bucket in October - Weekday', fontsize=14)
plt.xlabel('Time Bucket', fontsize=12)
plt.ylabel('Average Count', fontsize=12)

# Rotate the x-axis labels for readability
plt.xticks(rotation=45, ha='right')

# Show the plot
plt.tight_layout()
plt.show()

In [11]:
oct_wk_summ = oct_wk.groupby(["o_stop", "time_bucket"]).agg({
    "board": "sum",
    "alight": "sum",
    "latitude": "min",
    "longitude": "min"
}).reset_index()

In [17]:
oct_wk_summ['dummy_timestamp'] = '2022/04/01 '+ oct_wk_summ['time_bucket'] 

oct_wk_sedf = pd.DataFrame.spatial.from_xy(oct_wk_summ, x_column="longitude", y_column="latitude", sr=4326)

oct_wk_sedf

,o_stop,time_bucket,board,alight,latitude,longitude,dummy_timestamp,SHAPE
0,34,00:00:00,19,9,44.971011,-93.262521,2022/04/01 00:00:00,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
1,34,01:00:00,13,4,44.971011,-93.262521,2022/04/01 01:00:00,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
2,34,01:30:00,19,8,44.971011,-93.262521,2022/04/01 01:30:00,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
3,34,04:30:00,11,0,44.971011,-93.262521,2022/04/01 04:30:00,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
4,34,05:30:00,36,6,44.971011,-93.262521,2022/04/01 05:30:00,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
...,...,...,...,...,...,...,...,...
232908,90285,18:00:00,0,0,NaN,NaN,2022/04/01 18:00:00,None
232909,90285,20:00:00,0,0,NaN,NaN,2022/04/01 20:00:00,None
232910,90285,21:00:00,0,0,NaN,NaN,2022/04/01 21:00:00,None
232911,90285,21:30:00,0,0,NaN,NaN,2022/04/01 21:30:00,None


In [18]:
oct_wk_sedf.spatial.to_featureclass(location = r"C:\Users\KOlso\Documents\ArcGIS\Projects\SpatioTempporal Modeling Final Project\SpatioTempporal Modeling Final Project.gdb\oct_wk_summary")


'C:\\Users\\KOlso\\Documents\\ArcGIS\\Projects\\SpatioTempporal Modeling Final Project\\SpatioTempporal Modeling Final Project.gdb\\oct_wk_summary'

In [19]:
arcpy.management.ConvertTimeField(
    in_table="oct_wk_summary",
    input_time_field="dummy_timestamp",
    input_time_format="yyyy/MM/dd HH:mm:ss",
    output_time_field="dummy_timestamp_Converted",
    output_time_type="DATE",
    output_time_format="",
    timezone_or_field=""
)

<Result 'oct_wk_summary'>

In [20]:
arcpy.management.ApplySymbologyFromLayer(
    in_layer="oct_wk_summary",
    in_symbology_layer="apr_wk_summary",
    symbology_fields="VALUE_FIELD board board",
    update_symbology="MAINTAIN"
)

<Result 'oct_wk_summary'>

## October - routes 2 and 3

In [32]:
oct_wk_3 = oct_wk[oct_wk['route_abbr'] == 3].copy()
oct_wk_2 = oct_wk[oct_wk['route_abbr'] == 2].copy()

In [33]:
oct_wk_3 = oct_wk_3.groupby(["o_stop", "time_bucket"]).agg({
    "board": "sum",
    "alight": "sum",
    "latitude": "min",
    "longitude": "min"
}).reset_index()

oct_wk_2 = oct_wk_2.groupby(["o_stop", "time_bucket"]).agg({
    "board": "sum",
    "alight": "sum",
    "latitude": "min",
    "longitude": "min"
}).reset_index()

In [34]:
oct_wk_3['dummy_timestamp'] = '2022/04/01 '+ oct_wk_3['time_bucket']
oct_wk_2['dummy_timestamp'] = '2022/04/01 '+ oct_wk_2['time_bucket'] 


In [35]:
oct_wk_3_sedf = pd.DataFrame.spatial.from_xy(oct_wk_3, x_column="longitude", y_column="latitude", sr=4326)
oct_wk_2_sedf = pd.DataFrame.spatial.from_xy(oct_wk_2, x_column="longitude", y_column="latitude", sr=4326)

In [36]:
oct_wk_3_sedf.spatial.to_featureclass(location = r"C:\Users\KOlso\Documents\ArcGIS\Projects\SpatioTempporal Modeling Final Project\SpatioTempporal Modeling Final Project.gdb\oct_wk_3_summary")
oct_wk_2_sedf.spatial.to_featureclass(location = r"C:\Users\KOlso\Documents\ArcGIS\Projects\SpatioTempporal Modeling Final Project\SpatioTempporal Modeling Final Project.gdb\oct_wk_2_summary")


'C:\\Users\\KOlso\\Documents\\ArcGIS\\Projects\\SpatioTempporal Modeling Final Project\\SpatioTempporal Modeling Final Project.gdb\\oct_wk_2_summary'

## February - weekdays

In [15]:
#convert time to known time format
feb_df["time"] = feb_df["actual_arrival_time"].apply(lambda x: str(timedelta(seconds=x % 86400)))

feb_df

,board,alight,c_load,calendar_id,block_abbr,trip_id,route_abbr,line_type,o_stop,stop_sequence_number,service_abbr,route_direction_abbr,pattern_abbr,block_stop_order,latitude,longitude,actual_arrival_time,time
0,3,0,5,20230201,1380,1,54,Local,11861,5,WK,W,UNDPMOA200,6,44.947217,-93.093799,11132,3:05:32
1,0,0,5,20230201,1380,1,54,Local,11862,6,WK,W,UNDPMOA200,7,44.946197,-93.095645,11190,3:06:30
2,1,0,6,20230201,1380,1,54,Local,11863,7,WK,W,UNDPMOA200,8,44.946112,-93.097146,11258,3:07:38
3,0,0,6,20230201,1380,1,54,Local,3264,8,WK,W,UNDPMOA200,9,44.946248,-93.099029,11290,3:08:10
4,0,0,6,20230201,1380,1,54,Local,47485,9,WK,W,UNDPMOA200,10,44.945413,-93.102246,11330,3:08:50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6697581,0,0,1,20230228,1246,282,17,Local,5801,32,WK,E,BLCALAFR00,557,44.948441,-93.338612,93870,2:04:30
6697582,0,0,1,20230228,1246,282,17,Local,5802,33,WK,E,BLCALAFR00,558,44.948424,-93.336343,93882,2:04:42
6697583,0,0,1,20230228,1246,282,17,Local,5803,34,WK,E,BLCALAFR00,559,44.948450,-93.334141,93892,2:04:52
6697584,0,0,1,20230228,1246,282,17,Local,5804,35,WK,E,BLCALAFR00,560,44.948461,-93.332878,93900,2:05:00


In [16]:
# Apply the function to create a new column
feb_df['time_bucket'] = feb_df['time'].apply(round_to_next_30_minutes)

# Output the DataFrame to check the new column
feb_df

,board,alight,c_load,calendar_id,block_abbr,trip_id,route_abbr,line_type,o_stop,stop_sequence_number,service_abbr,route_direction_abbr,pattern_abbr,block_stop_order,latitude,longitude,actual_arrival_time,time,time_bucket
0,3,0,5,20230201,1380,1,54,Local,11861,5,WK,W,UNDPMOA200,6,44.947217,-93.093799,11132,3:05:32,03:30:00
1,0,0,5,20230201,1380,1,54,Local,11862,6,WK,W,UNDPMOA200,7,44.946197,-93.095645,11190,3:06:30,03:30:00
2,1,0,6,20230201,1380,1,54,Local,11863,7,WK,W,UNDPMOA200,8,44.946112,-93.097146,11258,3:07:38,03:30:00
3,0,0,6,20230201,1380,1,54,Local,3264,8,WK,W,UNDPMOA200,9,44.946248,-93.099029,11290,3:08:10,03:30:00
4,0,0,6,20230201,1380,1,54,Local,47485,9,WK,W,UNDPMOA200,10,44.945413,-93.102246,11330,3:08:50,03:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6697581,0,0,1,20230228,1246,282,17,Local,5801,32,WK,E,BLCALAFR00,557,44.948441,-93.338612,93870,2:04:30,02:30:00
6697582,0,0,1,20230228,1246,282,17,Local,5802,33,WK,E,BLCALAFR00,558,44.948424,-93.336343,93882,2:04:42,02:30:00
6697583,0,0,1,20230228,1246,282,17,Local,5803,34,WK,E,BLCALAFR00,559,44.948450,-93.334141,93892,2:04:52,02:30:00
6697584,0,0,1,20230228,1246,282,17,Local,5804,35,WK,E,BLCALAFR00,560,44.948461,-93.332878,93900,2:05:00,02:30:00


In [17]:
feb_wk = feb_df[feb_df['service_abbr'] == 'WK']
#feb_sat_sun = feb_df[feb_df['service_abbr'].isin(['SAT', 'SUN'])]

feb_wk

,board,alight,c_load,calendar_id,block_abbr,trip_id,route_abbr,line_type,o_stop,stop_sequence_number,service_abbr,route_direction_abbr,pattern_abbr,block_stop_order,latitude,longitude,actual_arrival_time,time,time_bucket
0,3,0,5,20230201,1380,1,54,Local,11861,5,WK,W,UNDPMOA200,6,44.947217,-93.093799,11132,3:05:32,03:30:00
1,0,0,5,20230201,1380,1,54,Local,11862,6,WK,W,UNDPMOA200,7,44.946197,-93.095645,11190,3:06:30,03:30:00
2,1,0,6,20230201,1380,1,54,Local,11863,7,WK,W,UNDPMOA200,8,44.946112,-93.097146,11258,3:07:38,03:30:00
3,0,0,6,20230201,1380,1,54,Local,3264,8,WK,W,UNDPMOA200,9,44.946248,-93.099029,11290,3:08:10,03:30:00
4,0,0,6,20230201,1380,1,54,Local,47485,9,WK,W,UNDPMOA200,10,44.945413,-93.102246,11330,3:08:50,03:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6697581,0,0,1,20230228,1246,282,17,Local,5801,32,WK,E,BLCALAFR00,557,44.948441,-93.338612,93870,2:04:30,02:30:00
6697582,0,0,1,20230228,1246,282,17,Local,5802,33,WK,E,BLCALAFR00,558,44.948424,-93.336343,93882,2:04:42,02:30:00
6697583,0,0,1,20230228,1246,282,17,Local,5803,34,WK,E,BLCALAFR00,559,44.948450,-93.334141,93892,2:04:52,02:30:00
6697584,0,0,1,20230228,1246,282,17,Local,5804,35,WK,E,BLCALAFR00,560,44.948461,-93.332878,93900,2:05:00,02:30:00


In [37]:
# Group by time_bucket and calendar_id, then count rows for each combination
feb_grouped = feb_wk.groupby(['time_bucket', 'calendar_id']).size().reset_index(name='count')

# Calculate the average count per time_bucket
feb_avg_counts = feb_grouped.groupby('time_bucket')['count'].mean()

# Sort the time_bucket values with 03:30:00 as the starting point
feb_avg_counts_sorted = feb_avg_counts.sort_index()  # Sort by time buckets
feb_reordered_avg_counts = pd.concat([
    feb_avg_counts_sorted.loc["03:30:00":],  # Start from 03:30:00 onward
    feb_avg_counts_sorted.loc[:"03:00:00"]   # Append 00:00:00 to 03:00:00 at the end
])

# Plot the reordered average histogram
plt.figure(figsize=(10, 6))
feb_reordered_avg_counts.plot(kind='bar', color='skyblue')

# Add title and labels for the chart
plt.title('Average Count of Stops per Time Bucket in February - Weekday', fontsize=14)
plt.xlabel('Time Bucket', fontsize=12)
plt.ylabel('Average Count', fontsize=12)

# Rotate the x-axis labels for readability
plt.xticks(rotation=45, ha='right')

# Show the plot
plt.tight_layout()
plt.show()

In [26]:
feb_wk_summ = feb_wk.groupby(["o_stop", "time_bucket"]).agg({
    "board": "sum",
    "alight": "sum",
    "latitude": "min",
    "longitude": "min"
}).reset_index()

In [31]:
feb_wk_summ['dummy_timestamp'] = '2022/04/01 '+ feb_wk_summ['time_bucket'] 

feb_wk_sedf = pd.DataFrame.spatial.from_xy(feb_wk_summ, x_column="longitude", y_column="latitude", sr=4326)

feb_wk_sedf

,o_stop,time_bucket,board,alight,latitude,longitude,dummy_timestamp,SHAPE
0,34,00:00:00,1,0,44.971011,-93.262521,2022/04/01 00:00:00,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
1,34,00:30:00,6,3,44.971011,-93.262521,2022/04/01 00:30:00,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
2,34,01:30:00,2,2,44.971011,-93.262521,2022/04/01 01:30:00,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
3,34,05:30:00,10,0,44.971011,-93.262521,2022/04/01 05:30:00,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
4,34,07:00:00,5,4,44.971011,-93.262521,2022/04/01 07:00:00,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
...,...,...,...,...,...,...,...,...
212373,90298,21:30:00,0,0,NaN,NaN,2022/04/01 21:30:00,None
212374,90298,22:00:00,1,1,NaN,NaN,2022/04/01 22:00:00,None
212375,90298,22:30:00,0,0,NaN,NaN,2022/04/01 22:30:00,None
212376,90298,23:00:00,0,0,NaN,NaN,2022/04/01 23:00:00,None


In [32]:
feb_wk_sedf.spatial.to_featureclass(location = r"C:\Users\KOlso\Documents\ArcGIS\Projects\SpatioTempporal Modeling Final Project\SpatioTempporal Modeling Final Project.gdb\feb_wk_summary")


'C:\\Users\\KOlso\\Documents\\ArcGIS\\Projects\\SpatioTempporal Modeling Final Project\\SpatioTempporal Modeling Final Project.gdb\\feb_wk_summary'

In [33]:
arcpy.management.ConvertTimeField(
    in_table="feb_wk_summary",
    input_time_field="dummy_timestamp",
    input_time_format="yyyy/MM/dd HH:mm:ss",
    output_time_field="dummy_timestamp_Converted",
    output_time_type="DATE",
    output_time_format="",
    timezone_or_field=""
)

<Result 'feb_wk_summary'>

In [34]:
arcpy.management.ApplySymbologyFromLayer(
    in_layer="feb_wk_summary",
    in_symbology_layer="apr_wk_summary",
    symbology_fields="VALUE_FIELD board board",
    update_symbology="MAINTAIN"
)

<Result 'feb_wk_summary'>

## February daily

In [49]:
feb_df

,board,alight,c_load,calendar_id,block_abbr,trip_id,route_abbr,line_type,o_stop,stop_sequence_number,service_abbr,route_direction_abbr,pattern_abbr,block_stop_order,latitude,longitude,actual_arrival_time,time,time_bucket
0,3,0,5,20230201,1380,1,54,Local,11861,5,WK,W,UNDPMOA200,6,44.947217,-93.093799,11132,3:05:32,03:30:00
1,0,0,5,20230201,1380,1,54,Local,11862,6,WK,W,UNDPMOA200,7,44.946197,-93.095645,11190,3:06:30,03:30:00
2,1,0,6,20230201,1380,1,54,Local,11863,7,WK,W,UNDPMOA200,8,44.946112,-93.097146,11258,3:07:38,03:30:00
3,0,0,6,20230201,1380,1,54,Local,3264,8,WK,W,UNDPMOA200,9,44.946248,-93.099029,11290,3:08:10,03:30:00
4,0,0,6,20230201,1380,1,54,Local,47485,9,WK,W,UNDPMOA200,10,44.945413,-93.102246,11330,3:08:50,03:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6697581,0,0,1,20230228,1246,282,17,Local,5801,32,WK,E,BLCALAFR00,557,44.948441,-93.338612,93870,2:04:30,02:30:00
6697582,0,0,1,20230228,1246,282,17,Local,5802,33,WK,E,BLCALAFR00,558,44.948424,-93.336343,93882,2:04:42,02:30:00
6697583,0,0,1,20230228,1246,282,17,Local,5803,34,WK,E,BLCALAFR00,559,44.948450,-93.334141,93892,2:04:52,02:30:00
6697584,0,0,1,20230228,1246,282,17,Local,5804,35,WK,E,BLCALAFR00,560,44.948461,-93.332878,93900,2:05:00,02:30:00


In [47]:
feb_daily = feb_df.copy()

feb_daily_summ = feb_daily.groupby(["o_stop", "calendar_id"]).agg({
    "board": "sum",
    "alight": "sum",
    "latitude": "min",
    "longitude": "min"
}).reset_index()

In [51]:
feb_daily_summ_sedf = pd.DataFrame.spatial.from_xy(feb_daily_summ, x_column="longitude", y_column="latitude", sr=4326)

feb_daily_summ_sedf

,o_stop,calendar_id,board,alight,latitude,longitude,SHAPE
0,34,20230201,43,19,44.971011,-93.262521,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
1,34,20230202,33,3,44.971011,-93.262521,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
2,34,20230203,26,9,44.971011,-93.262521,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
3,34,20230204,19,5,44.971011,-93.262521,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
4,34,20230205,12,3,44.971011,-93.262521,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
...,...,...,...,...,...,...,...
196136,90298,20230224,3,1,NaN,NaN,None
196137,90298,20230225,0,0,NaN,NaN,None
196138,90298,20230226,7,4,NaN,NaN,None
196139,90298,20230227,0,0,NaN,NaN,None


In [52]:
feb_daily_summ_sedf.spatial.to_featureclass(location = r"C:\Users\KOlso\Documents\ArcGIS\Projects\SpatioTempporal Modeling Final Project\SpatioTempporal Modeling Final Project.gdb\feb_daily_summ")

'C:\\Users\\KOlso\\Documents\\ArcGIS\\Projects\\SpatioTempporal Modeling Final Project\\SpatioTempporal Modeling Final Project.gdb\\feb_daily_summ'

## October - weekend

In [9]:
oct_df["time"] = oct_df["actual_arrival_time"].apply(lambda x: str(timedelta(seconds=x % 86400)))

oct_df

,board,alight,c_load,calendar_id,block_abbr,trip_id,route_abbr,line_type,o_stop,stop_sequence_number,service_abbr,route_direction_abbr,pattern_abbr,block_stop_order,latitude,longitude,actual_arrival_time,time
0,0,0,2,20221001,1281,1,54,Local,10358,11,SAT,W,UNDPMOA200,12,44.942673,-93.105112,11454,3:10:54
1,0,0,2,20221001,1281,1,54,Local,10359,12,SAT,W,UNDPMOA200,13,44.941437,-93.106737,11470,3:11:10
2,0,0,2,20221001,1281,1,54,Local,10366,13,SAT,W,UNDPMOA200,14,44.934241,-93.117310,11554,3:12:34
3,0,0,2,20221001,1281,1,54,Local,11698,14,SAT,W,UNDPMOA200,15,44.926972,-93.127943,11672,3:14:32
4,0,0,2,20221001,1281,1,54,Local,13117,15,SAT,W,UNDPMOA200,16,44.923360,-93.133283,11714,3:15:14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7969556,0,3,-3,20221031,1066,370,3,Local,57018,65,WK,E,WALA5CED00,407,NaN,NaN,94390,2:13:10
7969557,0,0,-3,20221031,1066,370,3,Local,49450,66,WK,E,WALA5CED00,408,44.950081,-93.100056,94498,2:14:58
7969558,0,0,-3,20221031,1066,370,3,Local,11912,67,WK,E,WALA5CED00,409,44.950506,-93.097629,94538,2:15:38
7969559,0,0,-3,20221031,1066,370,3,Local,20013,68,WK,E,WALA5CED00,410,44.947507,-93.094443,94626,2:17:06


In [14]:
oct_df['time_bucket'] = oct_df['time'].apply(round_to_next_30_minutes)

In [15]:
oct_wkd = oct_df[oct_df['service_abbr'].isin(['SAT', 'SUN'])]

In [16]:
oct_wkd_summ = oct_wkd.groupby(["o_stop", "time_bucket"]).agg({
    "board": "mean",
    "alight": "mean",
    "latitude": "min",
    "longitude": "min"
}).reset_index()

oct_wkd_summ

,o_stop,time_bucket,board,alight,latitude,longitude
0,34,00:00:00,1.000000,0.500000,44.971011,-93.262521
1,34,00:30:00,0.428571,0.285714,44.971011,-93.262521
2,34,01:00:00,0.750000,0.250000,44.971011,-93.262521
3,34,01:30:00,1.000000,0.222222,44.971011,-93.262521
4,34,04:30:00,0.000000,0.400000,44.971011,-93.262521
...,...,...,...,...,...,...
191542,90285,19:00:00,0.000000,0.000000,NaN,NaN
191543,90285,20:00:00,0.000000,0.000000,NaN,NaN
191544,90285,20:30:00,0.000000,0.000000,NaN,NaN
191545,90285,21:00:00,0.000000,0.000000,NaN,NaN


In [17]:
oct_wkd_summ['dummy_timestamp'] = '2022/04/01 '+ oct_wkd_summ['time_bucket'] 

oct_wkd_sedf = pd.DataFrame.spatial.from_xy(oct_wkd_summ, x_column="longitude", y_column="latitude", sr=4326)

oct_wkd_sedf.spatial.to_featureclass(location = r"C:\Users\KOlso\Documents\ArcGIS\Projects\SpatioTempporal Modeling Final Project\SpatioTempporal Modeling Final Project.gdb\oct_wkd_summary")

'C:\\Users\\KOlso\\Documents\\ArcGIS\\Projects\\SpatioTempporal Modeling Final Project\\SpatioTempporal Modeling Final Project.gdb\\oct_wkd_summary'

In [35]:
apr_sat_sun_summ = apr_sat_sun.groupby(["o_stop", "time_bucket"]).agg({
    "board": "mean",
    "alight": "mean",
    "latitude": "min",
    "longitude": "min"
}).reset_index()

apr_sat_sun_summ

,o_stop,time_bucket,board,alight,latitude,longitude
0,34,00:00:00,1.200000,0.600000,44.971011,-93.262521
1,34,00:30:00,0.800000,0.400000,44.971011,-93.262521
2,34,01:00:00,1.000000,0.750000,44.971011,-93.262521
3,34,01:30:00,0.571429,0.285714,44.971011,-93.262521
4,34,04:30:00,0.285714,0.142857,44.971011,-93.262521
...,...,...,...,...,...,...
194936,90240,21:30:00,0.000000,0.000000,NaN,NaN
194937,90240,22:00:00,0.000000,0.000000,NaN,NaN
194938,90240,22:30:00,0.000000,0.000000,NaN,NaN
194939,90240,23:00:00,0.000000,0.000000,NaN,NaN


In [39]:
#add date field to make next step easier - arcGIS Pro seems to need a date to go with the time - convert to sedf

#also - enable time in layer

apr_sat_sun_summ['dummy_timestamp'] = '2022/04/01 '+ apr_sat_sun_summ['time_bucket'] 

apr_sat_sun_sedf = pd.DataFrame.spatial.from_xy(apr_sat_sun_summ, x_column="longitude", y_column="latitude", sr=4326)

apr_sat_sun_sedf

,o_stop,time_bucket,board,alight,latitude,longitude,dummy_timestamp,SHAPE
0,34,00:00:00,1.200000,0.600000,44.971011,-93.262521,2022/04/01 00:00:00,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
1,34,00:30:00,0.800000,0.400000,44.971011,-93.262521,2022/04/01 00:30:00,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
2,34,01:00:00,1.000000,0.750000,44.971011,-93.262521,2022/04/01 01:00:00,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
3,34,01:30:00,0.571429,0.285714,44.971011,-93.262521,2022/04/01 01:30:00,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
4,34,04:30:00,0.285714,0.142857,44.971011,-93.262521,2022/04/01 04:30:00,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
...,...,...,...,...,...,...,...,...
194936,90240,21:30:00,0.000000,0.000000,NaN,NaN,2022/04/01 21:30:00,None
194937,90240,22:00:00,0.000000,0.000000,NaN,NaN,2022/04/01 22:00:00,None
194938,90240,22:30:00,0.000000,0.000000,NaN,NaN,2022/04/01 22:30:00,None
194939,90240,23:00:00,0.000000,0.000000,NaN,NaN,2022/04/01 23:00:00,None


In [40]:
apr_sat_sun_sedf.spatial.to_featureclass(location = r"C:\Users\KOlso\Documents\ArcGIS\Projects\SpatioTempporal Modeling Final Project\SpatioTempporal Modeling Final Project.gdb\apr_sat_sun_summary")

#don't forget to enable time in the layer

'C:\\Users\\KOlso\\Documents\\ArcGIS\\Projects\\SpatioTempporal Modeling Final Project\\SpatioTempporal Modeling Final Project.gdb\\apr_sat_sun_summary'

In [ ]:
#optional - can do by hand as well

arcpy.management.ConvertTimeField(
    in_table="apr_sat_sun_summary",
    input_time_field="dummy_timestamp",
    input_time_format="yyyy/MM/dd HH:mm:ss",
    output_time_field="dummy_timestamp_Converted",
    output_time_type="DATE",
    output_time_format="",
    timezone_or_field=""
)

#### Make route specific animations

In [50]:
apr_wk_3 = apr_wk[apr_wk['route_abbr'] == 3].copy()
apr_wk_2 = apr_wk[apr_wk['route_abbr'] == 2].copy()

apr_wk_3

,board,alight,c_load,calendar_id,block_abbr,trip_id,route_abbr,line_type,o_stop,stop_sequence_number,service_abbr,route_direction_abbr,pattern_abbr,block_stop_order,latitude,longitude,actual_arrival_time,time,time_bucket
291,0,0,0,20220401,1027,2,3,Urb Loc,16342,1,WK,E,WA10UNDP00,2,44.990252,-93.280273,14222,3:57:02,04:00:00
292,0,0,0,20220401,1027,2,3,Urb Loc,16406,2,WK,E,WA10UNDP00,3,44.990421,-93.280799,14622,4:03:42,04:30:00
293,0,0,0,20220401,1027,2,3,Urb Loc,57010,3,WK,E,WA10UNDP00,4,NaN,NaN,14656,4:04:16,04:30:00
294,0,0,0,20220401,1027,2,3,Urb Loc,57011,4,WK,E,WA10UNDP00,5,NaN,NaN,14686,4:04:46,04:30:00
295,0,0,0,20220401,1027,2,3,Urb Loc,17876,5,WK,E,WA10UNDP00,6,44.984703,-93.272634,14718,4:05:18,04:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7888202,1,1,7,20220429,1621,370,3,Urb Loc,57018,65,WK,E,WA105CED00,551,NaN,NaN,94410,2:13:30,02:30:00
7888203,0,1,6,20220429,1621,370,3,Urb Loc,49450,66,WK,E,WA105CED00,552,44.950081,-93.100056,94516,2:15:16,02:30:00
7888204,0,0,6,20220429,1621,370,3,Urb Loc,11912,67,WK,E,WA105CED00,553,44.950506,-93.097629,94598,2:16:38,02:30:00
7888205,0,0,6,20220429,1621,370,3,Urb Loc,20013,68,WK,E,WA105CED00,554,44.947507,-93.094443,94696,2:18:16,02:30:00


In [51]:
apr_wk_3 = apr_wk_3.groupby(["o_stop", "time_bucket"]).agg({
    "board": "mean",
    "alight": "mean",
    "latitude": "min",
    "longitude": "min"
}).reset_index()

apr_wk_3

,o_stop,time_bucket,board,alight,latitude,longitude
0,826,00:30:00,0.000000,0.000000,44.960589,-93.108782
1,826,01:00:00,0.000000,0.000000,44.960589,-93.108782
2,826,05:30:00,0.000000,0.000000,44.960589,-93.108782
3,826,06:30:00,0.000000,0.000000,44.960589,-93.108782
4,826,07:00:00,0.000000,0.000000,44.960589,-93.108782
...,...,...,...,...,...,...
7200,80289,21:30:00,0.153846,0.307692,NaN,NaN
7201,80289,22:00:00,0.100000,0.450000,NaN,NaN
7202,80289,22:30:00,0.350000,0.200000,NaN,NaN
7203,80289,23:00:00,0.000000,0.105263,NaN,NaN


In [46]:
apr_wk_2 = apr_wk_2.groupby(["o_stop", "time_bucket"]).agg({
    "board": "mean",
    "alight": "mean",
    "latitude": "min",
    "longitude": "min"
}).reset_index()

apr_wk_2

,o_stop,time_bucket,board,alight,latitude,longitude
0,1096,00:00:00,0.000000,1.000000,44.962379,-93.29183
1,1096,00:30:00,0.062500,0.937500,44.962379,-93.29183
2,1096,01:00:00,0.047619,0.333333,44.962379,-93.29183
3,1096,01:30:00,0.095238,0.428571,44.962379,-93.29183
4,1096,06:00:00,0.000000,3.904762,44.962379,-93.29183
...,...,...,...,...,...,...
2939,80290,21:30:00,0.621622,0.216216,NaN,NaN
2940,80290,22:00:00,0.526316,0.526316,NaN,NaN
2941,80290,22:30:00,0.050000,0.650000,NaN,NaN
2942,80290,23:00:00,0.200000,0.350000,NaN,NaN


In [52]:
apr_wk_3['dummy_timestamp'] = '2022/04/01 '+ apr_wk_3['time_bucket'] 

apr_wk_3_sedf = pd.DataFrame.spatial.from_xy(apr_wk_3, x_column="longitude", y_column="latitude", sr=4326)

apr_wk_3_sedf

,o_stop,time_bucket,board,alight,latitude,longitude,dummy_timestamp,SHAPE
0,826,00:30:00,0.000000,0.000000,44.960589,-93.108782,2022/04/01 00:30:00,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
1,826,01:00:00,0.000000,0.000000,44.960589,-93.108782,2022/04/01 01:00:00,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
2,826,05:30:00,0.000000,0.000000,44.960589,-93.108782,2022/04/01 05:30:00,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
3,826,06:30:00,0.000000,0.000000,44.960589,-93.108782,2022/04/01 06:30:00,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
4,826,07:00:00,0.000000,0.000000,44.960589,-93.108782,2022/04/01 07:00:00,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
...,...,...,...,...,...,...,...,...
7200,80289,21:30:00,0.153846,0.307692,NaN,NaN,2022/04/01 21:30:00,None
7201,80289,22:00:00,0.100000,0.450000,NaN,NaN,2022/04/01 22:00:00,None
7202,80289,22:30:00,0.350000,0.200000,NaN,NaN,2022/04/01 22:30:00,None
7203,80289,23:00:00,0.000000,0.105263,NaN,NaN,2022/04/01 23:00:00,None


In [48]:
apr_wk_2['dummy_timestamp'] = '2022/04/01 '+ apr_wk_2['time_bucket'] 

apr_wk_2_sedf = pd.DataFrame.spatial.from_xy(apr_wk_2, x_column="longitude", y_column="latitude", sr=4326)

apr_wk_2_sedf

,o_stop,time_bucket,board,alight,latitude,longitude,dummy_timestamp,SHAPE
0,1096,00:00:00,0.000000,1.000000,44.962379,-93.29183,2022/04/01 00:00:00,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
1,1096,00:30:00,0.062500,0.937500,44.962379,-93.29183,2022/04/01 00:30:00,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
2,1096,01:00:00,0.047619,0.333333,44.962379,-93.29183,2022/04/01 01:00:00,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
3,1096,01:30:00,0.095238,0.428571,44.962379,-93.29183,2022/04/01 01:30:00,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
4,1096,06:00:00,0.000000,3.904762,44.962379,-93.29183,2022/04/01 06:00:00,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
...,...,...,...,...,...,...,...,...
2939,80290,21:30:00,0.621622,0.216216,NaN,NaN,2022/04/01 21:30:00,None
2940,80290,22:00:00,0.526316,0.526316,NaN,NaN,2022/04/01 22:00:00,None
2941,80290,22:30:00,0.050000,0.650000,NaN,NaN,2022/04/01 22:30:00,None
2942,80290,23:00:00,0.200000,0.350000,NaN,NaN,2022/04/01 23:00:00,None


In [53]:
apr_wk_3_sedf.spatial.to_featureclass(location = r"C:\Users\KOlso\Documents\ArcGIS\Projects\SpatioTempporal Modeling Final Project\SpatioTempporal Modeling Final Project.gdb\apr_wk_3_summary")
apr_wk_2_sedf.spatial.to_featureclass(location = r"C:\Users\KOlso\Documents\ArcGIS\Projects\SpatioTempporal Modeling Final Project\SpatioTempporal Modeling Final Project.gdb\apr_wk_2_summary")


'C:\\Users\\KOlso\\Documents\\ArcGIS\\Projects\\SpatioTempporal Modeling Final Project\\SpatioTempporal Modeling Final Project.gdb\\apr_wk_3_summary'

In [ ]:
#pivot table to convert to stop-based vectors - can be used to create ST clusters
apr_wk_board_pivot = pd.pivot(apr_wk_summ, values = "board", index = "o_stop", columns = "time_bucket")
apr_wk_board_pivot

In [ ]:
#fill nulls with 0s
apr_wk_board_pivot = apr_wk_board_pivot.fillna(0)

#create numpy array of values
time_series_columns = [col for col in apr_wk_board_pivot.columns if ":" in col]
time_series = apr_wk_board_pivot[time_series_columns].values

In [ ]:
#show array to confirm
times_series

In [ ]:
from dtaidistance import dtw
import numpy as np

ds = dtw.distance_matrix_fast(time_series)

from sklearn.cluster import spectralClustering
clustering = spectralClustering(

)

In [18]:
# Define a function to calculate reordered average counts
def calculate_reordered_avg_counts(df, start_time="03:30:00"):
    grouped = df.groupby(['time_bucket', 'calendar_id']).size().reset_index(name='count')
    avg_counts = grouped.groupby('time_bucket')['count'].mean()
    avg_counts_sorted = avg_counts.sort_index()  # Sort by time buckets
    reordered_avg_counts = pd.concat([
        avg_counts_sorted.loc[start_time:],  # Start from the specified time onward
        avg_counts_sorted.loc[:start_time]   # Append earlier times at the end
    ])
    return reordered_avg_counts

# Calculate reordered average counts for each dataframe
apr_avg_counts = calculate_reordered_avg_counts(apr_wk)
jul_avg_counts = calculate_reordered_avg_counts(jul_wk)
oct_avg_counts = calculate_reordered_avg_counts(oct_wk)
feb_avg_counts = calculate_reordered_avg_counts(feb_wk)

# Plot the line graph
plt.figure(figsize=(12, 8))

# Plot each dataset with a unique color and label
plt.plot(apr_avg_counts, label='April', color='skyblue', marker='o')
plt.plot(jul_avg_counts, label='July', color='orange', marker='o')
plt.plot(oct_avg_counts, label='October', color='green', marker='o')
plt.plot(feb_avg_counts, label='February', color='purple', marker='o')

# Add title and labels
plt.title('Average Count of Stops per Time Bucket (Weekday)', fontsize=16)
plt.xlabel('Time Bucket', fontsize=14)
plt.ylabel('Average Count', fontsize=14)

# Rotate the x-axis labels for readability
plt.xticks(rotation=45, ha='right')

# Add a legend
plt.legend(title="Month", fontsize=12)

# Show the plot
plt.tight_layout()
plt.show()
